In [1]:
from collections import Counter
import itertools
from tqdm import tqdm

import numpy as np
from scipy.spatial import distance


def get_kmers(dna, k):
    return [np.array(list(dna[i : i + k]))
            for i in range(len(dna) - k)]


def get_base_counts(dna):
    bases = ['A', 'T', 'G', 'C']
    return np.array([
        np.sum(dna == base) 
        for base in bases
    ])


def get_base_pseudocounts(dna):
    return get_base_counts(dna) + 1


base_to_num = {
    'A' : 0, 'T' : 1,
    'G' : 2, 'C' : 3
}

num_to_base = {
    val : key
    for key, val in base_to_num.items()
}


def get_profile(motifs):
    motif_mx = np.vstack(motifs)
    t, k = motif_mx.shape
    counts = np.column_stack([
        get_base_pseudocounts(col)
        for col in motif_mx.T
    ])
    assert counts.shape == (4, k)
    return counts / counts.sum(axis=0)


def eval_kmer_proba(kmer, profile):
    return np.prod([
        profile[base_to_num[kmer[i]], i]    
        for i in range(len(kmer))
    ])


def find_most_probable_motif(dna, k, profile):
    kmers_with_probas = np.vstack([
        (kmer, eval_kmer_proba(kmer, profile))
        for kmer in get_kmers(dna, k)
    ])
    top_row = np.argmax(kmers_with_probas[:, 1])
    return kmers_with_probas[top_row, 0]


def get_consensus(profile):
    return np.array([
        num_to_base[np.argmax(col)]
        for col in profile.T
    ])


def get_score(motifs):
    motif_mx = np.vstack(motifs)
    profile = get_profile(motif_mx)
    consensus = get_consensus(profile)
    return np.sum(
        (motif != consensus).sum()
        for motif in motif_mx
    )


def randomized_motif_search(reads, k, t):
    motifs = []
    for i in range(t):
        dna = reads[i]
        start = np.random.randint(low=0, high=n-k)
        motifs.append(np.array(list(dna[start : start + k])))
    best_motifs = motifs
    while True:
        profile = get_profile(motifs)
        motifs = [
            find_most_probable_motif(dna, k, profile)
            for dna in reads
        ]
        if get_score(motifs) < get_score(best_motifs):
            best_motifs = motifs
        else: 
            return best_motifs

In [2]:
lines = [line.strip() for line in list(open("randomized_motif_search.txt", "r"))]
#target_motifs = [np.array(list(line.strip())) for line in list(open("target.txt", "r"))]
k, t = (int(num) for num in lines[0].split())
reads = lines[1:]
n = len(reads[0])

MC_ITERS = 1000
best_motifs = [ 
    np.array(list(dna[:k]))
    for dna in reads
]  
best_score = get_score(best_motifs)
for _ in tqdm(range(MC_ITERS), desc="monte-carlo iterations passed"):
    candidate_motifs = randomized_motif_search(reads, k, t)
    candidate_score = get_score(candidate_motifs)
    if candidate_score < best_score:
        best_score = candidate_score
        best_motifs = candidate_motifs
        print(best_score)

#    print("TARGET: ", get_score(target_motifs))

answers = '\n'.join([''.join(motif) for motif in best_motifs])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:77: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
monte-carlo iterations passed:   0%|          | 0/1000 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
monte-carlo iterations passed:   0%|          | 1/1000 [00:00<11:17,  1.47it/s]

129


monte-carlo iterations passed:   0%|          | 2/1000 [00:01<10:31,  1.58it/s]

64


monte-carlo iterations passed:   4%|▎         | 35/1000 [00:19<13:21,  1.20it/s]

63


monte-carlo iterations passed:  53%|█████▎    | 528/1000 [05:34<04:59,  1.58it/s]


KeyboardInterrupt: 

In [11]:
print(answers)

CATGGGGAAAACTGA
CCTCTCGATCACCGA
CCTATAGATCACCGA
CCGATTGATCACCGA
CCTTGTGCAGACCGA
CCTTGCCTTCACCGA
CCTTGTTGCCACCGA
ACTTGTGATCACCTT
CCTTGTGATCAATTA
CCTTGTGATCTGTGA
CCTTGTGATCACTCC
AACTGTGATCACCGA
CCTTAGTATCACCGA
CCTTGTGAAATCCGA
CCTTGTCGCCACCGA
TGTTGTGATCACCGC
CACCGTGATCACCGA
CCTTGGTTTCACCGA
CCTTTGCATCACCGA
CCTTGTGATTTACGA


In [ ]:
CATGGGGAAAACTGA
CCTCTCGATCACCGA
CCTATAGATCACCGA
CCGATTGATCACCGA
CCTTGTGCAGACCGA
CCTTGCCTTCACCGA
CCTTGTTGCCACCGA
ACTTGTGATCACCTT
CCTTGTGATCAATTA
CCTTGTGATCTGTGA
CCTTGTGATCACTCC
AACTGTGATCACCGA
CCTTAGTATCACCGA
CCTTGTGAAATCCGA
CCTTGTCGCCACCGA
TGTTGTGATCACCGC
CACCGTGATCACCGA
CCTTGGTTTCACCGA
CCTTTGCATCACCGA
CCTTGTGATTTACGA